In [121]:
import pandas as pd
import numpy as np
banpo_power = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/A365. 반포써밋2 발전량.csv", encoding="ms949")
# banpo_weather = pd.read_csv("./A365. 반포써밋2 환경센서.csv", encoding="ms949")
LH_power = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/A569. LH서울공릉 행복주택 발전량.csv", encoding="ms949")
# LH_weather = pd.read_csv("./A569. LH서울공릉 행복주택 환경센서.csv", encoding="ms949")
ongok_power = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/A643. 온곡중 체육관 발전량.csv", encoding="ms949")
# ongok_weather = pd.read_csv("./A643. 온곡중 체육관 환경센서.csv", encoding="ms949")
jumonjin_power = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/B115. 주문진해안주차타워 발전량.csv", encoding="ms949")

banpo_power = banpo_power[banpo_power["pow_cap"] == 20 ]
banpo_power
# jumonjin_weather = pd.read_csv("./B115. 주문진해안주차타워 환경센서.csv", encoding="ms949")

,pow_index,pow_id,pow_date,pow_time,pow_cap,pow_dcv,pow_dca,pow_dcp,pow_acvr,pow_acvs,pow_acvt,pow_acar,pow_acas,pow_acat,pow_acp,pow_pf,pow_totpower,pow_freq
0,520128,1,2020-10-23,6:58:21,20,521,0.0,0,377,375,377,1.8,1.8,1.8,0,0.0,43886000,59.9
1,520129,1,2020-10-23,7:03:15,20,403,0.0,0,378,376,376,2.0,1.9,1.9,100,0.0,43886000,60.0
3,520131,1,2020-10-23,7:08:07,20,360,0.2,0,377,375,375,1.8,1.9,1.9,200,0.0,43886000,59.9
5,520133,1,2020-10-23,7:12:59,20,364,0.3,0,374,373,373,1.7,1.9,1.8,200,0.0,43886000,60.0
7,520135,1,2020-10-23,7:17:50,20,446,0.2,0,377,376,377,1.8,1.8,2.0,300,0.0,43886000,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,524723,1,2020-11-09,17:03:07,20,384,0.0,0,378,376,377,1.5,1.7,1.6,0,0.0,44758000,60.0
4597,524725,1,2020-11-09,17:12:52,20,457,0.0,0,380,377,378,1.6,1.7,1.8,0,0.0,44758000,60.0
4598,524726,1,2020-11-09,17:17:46,20,435,0.0,0,378,376,377,1.7,1.8,1.7,0,0.0,44758000,60.0
4599,524727,1,2020-11-09,17:22:42,20,423,0.0,0,378,377,378,1.7,1.7,1.8,0,0.0,44758000,60.0


In [122]:
def groupby_solar_reset(ent_data_solar):
    ## 필요한 컬럼만 가져오기
    ent_data_need = ent_data_solar[["pow_date", "pow_time", "pow_dcv", "pow_dca"]]

    ## time 구분
    dummy_time_list = []
    for i in ent_data_need["pow_time"]:
        x =i.split(":")
        dummy_time_list.append(x[0])
    ent_data_need["dummy_time"] = dummy_time_list

    ent_data_need["watt"] = ent_data_need["pow_dcv"] * ent_data_need["pow_dca"]

    ## RESET 발전량 :: 직류 전압 X 직류 전류 : 발전량 ,, GROUPBY해서 직류X전류 의 평균량으로 구한다
    ent_data_perhour = ent_data_need.groupby(["pow_date", "dummy_time"]).mean()
    ent_data_perhour = ent_data_perhour.reset_index()


    drop_data = ent_data_perhour.isin({"dummy_time" : ["0", "1", "2", "3", "4", "5", "6", "7","8","18", "19", "20", "21", "22", "23"]})
    ent_data_perhour_sort = ent_data_perhour[~drop_data].dropna()

    return ent_data_perhour_sort

In [125]:
banpo_power_need = groupby_solar_reset(banpo_power)
banpo_power_need_day = banpo_power_need.groupby("pow_date").sum()[:-1]
banpo_power_need_day["watt"] = banpo_power_need_day["watt"] / 20
banpo_power_need_day
# banpo_power_need_day["watt"] = 

,pow_dcv,pow_dca,watt
pow_date,,,
2020-10-23,4498.151282,127.750769,3292.047859
2020-10-24,4478.993590,125.929359,3233.336974
2020-10-25,4385.019231,102.048718,2526.178269
2020-10-26,4379.711538,105.090385,2605.827067
2020-10-27,4383.737179,95.133974,2363.735769
2020-10-28,4313.927350,116.071154,2865.826111
2020-10-29,3896.525641,86.689103,2158.214231
2020-10-30,4362.160256,109.091667,2723.250128
2020-10-31,4394.474359,78.852198,1961.101703


In [80]:
def split_datetime(df):
    a = df[df["dummy_time"] == "09"]
    b = df[df["dummy_time"] == "10"]
    c = df[df["dummy_time"] == "11"]
    d = df[df["dummy_time"] == "12"]
    e = df[df["dummy_time"] == "13"]
    f = df[df["dummy_time"] == "14"]
    g = df[df["dummy_time"] == "15"]
    h = df[df["dummy_time"] == "16"]
    i = df[df["dummy_time"] == "17"]

    return a, b, c, d, e, f, g, h, i 



,pow_date,dummy_time,pow_dcv,pow_dca,watt
11,2020-10-23,9,527.333333,7.037500,3707.008333
22,2020-10-24,9,525.666667,6.729167,3533.504167
33,2020-10-25,9,513.875000,6.620833,3400.191667
45,2020-10-26,9,515.346154,5.665385,2905.923077
56,2020-10-27,9,512.923077,5.046154,2577.200000
67,2020-10-28,9,512.208333,6.158333,3155.220833
77,2020-10-29,9,512.583333,6.100000,3134.837500
88,2020-10-30,9,517.666667,5.641667,2935.720833
99,2020-10-31,9,507.208333,3.195833,1620.237500


In [74]:
new_weather_data = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/test_weather.csv", encoding="ms949")
new_weather_data_seoul = new_weather_data[new_weather_data["지점명"] == "서울"] 
new_weather_data_gang = new_weather_data[new_weather_data["지점명"] == "강릉"]
new_weather_data_seoul["match_solar"] = new_weather_data_seoul["일사(MJ/m2)"] * 1000000 / 3600
new_weather_data_gang["match_solar"] = new_weather_data_seoul["일사(MJ/m2)"] * 1000000 / 3600

new_weather_data_seoul

dummy_date = []
for i in new_weather_data_seoul["일시"]:
    dummy_date.append(i[11:13])

gang_dummy = []
for i in new_weather_data_gang["일시"]:
    # print(type(i))
    gang_dummy.append(i[10:13])
new_weather_data_seoul["dummy_time"] = dummy_date
new_weather_data_gang["dummy_time"] = gang_dummy
new_weather_data_seoul.reset_index()
new_weather_data_gang.reset_index()

drop_data = new_weather_data_seoul.isin({"dummy_time" : ["00", "01", "02", "03", "04", "05", "06", "07","08","18", "19", "20", "21", "22", "23"]})
drop_data_gang = new_weather_data_gang.isin({"dummy_time" : ["00", "01", "02", "03", "04", "05", "06", "07","08","18", "19", "20", "21", "22", "23"]})


new_select_seoul = new_weather_data_seoul[~drop_data].dropna()
new_select_gang = new_weather_data_gang[~drop_data_gang].dropna()

new_select_seoul


,지점,지점명,일시,일사(MJ/m2),match_solar,dummy_time
206,108,서울,2020-10-23 09:00,1.01,280.555556,09
207,108,서울,2020-10-23 10:00,1.68,466.666667,10
208,108,서울,2020-10-23 11:00,2.20,611.111111,11
209,108,서울,2020-10-23 12:00,2.48,688.888889,12
210,108,서울,2020-10-23 13:00,2.57,713.888889,13
...,...,...,...,...,...,...
397,108,서울,2020-11-08 13:00,2.36,655.555556,13
398,108,서울,2020-11-08 14:00,2.20,611.111111,14
399,108,서울,2020-11-08 15:00,1.80,500.000000,15
400,108,서울,2020-11-08 16:00,1.23,341.666667,16


In [82]:
a, b, c, d, e, f, g, h, i  = split_datetime(new_select_seoul)
a["reset_match"] = a["match_solar"] *1.4700
b["reset_match"] = b["match_solar"] *1.2674
c["reset_match"] = c["match_solar"] *1.1316
d["reset_match"] = d["match_solar"] *1.0937
e["reset_match"] = e["match_solar"] *1.0188
f["reset_match"] = f["match_solar"] *0.9105
g["reset_match"] = g["match_solar"] *0.8180
h["reset_match"] = h["match_solar"] *0.5019
i["reset_match"] = i["match_solar"] *0.1751


In [111]:
# test_banpo_reset_match = pd.concat([a, b, c, d, e, f, g, h, i])
test_banpo_reset_match

date_list = []
for i in test_banpo_reset_match["일시"]:
    date_list.append(i[:10])

test_banpo_reset_match["date"] = date_list
test_banpo_reset_match_perday = test_banpo_reset_match.groupby("date").sum()
test_banpo_reset_match_perday

,지점,일사(MJ/m2),match_solar,reset_match
date,,,,
2020-10-23,972,16.57,4602.777778,4490.702778
2020-10-24,972,16.25,4513.888889,4400.063611
2020-10-25,972,13.48,3744.444444,3782.352222
2020-10-26,972,13.43,3730.555556,3638.942222
2020-10-27,972,12.01,3336.111111,3338.226667
2020-10-28,972,15.00,4166.666667,4061.744444
2020-10-29,972,13.45,3736.111111,3787.304167
2020-10-30,972,14.35,3986.111111,3907.699722
2020-10-31,972,10.81,3002.777778,3013.494167


In [135]:
test_weather = pd.read_csv("./ENITT_SOLAR_DATA/ENITT_SOLAR_DATA/validation_data/weather_test_data.csv", encoding="ms949")
test_weather_seoul = test_weather[test_weather["지점명"] == "서울"]
test_weather_gang = test_weather[test_weather["일시"] == "강릉"]

# test_weather_seoul


test_weather_seoul_need = test_weather_seoul.drop(["지점", "지점명"], axis=1)
test_weather_seoul_need.index = test_weather_seoul_need["일시"]
# test_weather_seoul_need
# all_test_weather_banpo
all_test_weather_banpo = test_weather_seoul_need.join(test_banpo_reset_match_perday)
all_test_weather_banpo = all_test_weather_banpo[:-1]
# all_test_weather_banpo.to_csv("new_banpo_test.csv", encoding="ms949")
all_test_weather_solar_banpo = all_test_weather_banpo.join(banpo_power_need_day)
all_test_weather_solar_banpo_need = all_test_weather_solar_banpo[["최저기온(°C)", "최고기온(°C)", "일강수량(mm)", "평균 풍속(m/s)", "평균 상대습도(%)", "가조시간(hr)","평균 전운량(1/10)", "reset_match", "watt"]]
all_test_weather_solar_banpo_need.columns = ["최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량", "reset_match", "watt"]

all_test_weather_solar_banpo_need.fillna(0, inplace=True)

all_test_weather_solar_banpo_need.to_csv("test_data_banpo.csv", encoding="utf-8")